In [3]:
import pandas as pd
import plotly.graph_objects as go

# Load original data
overview_df = pd.read_csv("../overview_files_archive/overview_margin.csv")
overview_df['Date'] = pd.to_datetime(overview_df['Date'])

# Load financial events data
events_df = pd.read_csv("amenity_extractions.csv")
events_df['Financial Events Publication Date'] = pd.to_datetime(events_df['Financial Events Publication Date'])

# Rename columns for easier manipulation
events_df.rename(columns={
    'Financial Events Publication Date': 'Date',
    'Company Info Company': 'Company Ticker',
    'Financial Events Polarity': 'Polarity'
}, inplace=True)

# Calculate sentiment score for each date and company
polarity_count = events_df.groupby(['Date', 'Company Ticker', 'Polarity']).size().unstack(fill_value=0)
polarity_count['Sentiment Score'] = (polarity_count.get('pos', 0) - polarity_count.get('neg', 0)) / \
                                    (polarity_count.get('pos', 0) + polarity_count.get('neg', 0) + 1)

polarity_count.reset_index(inplace=True)

# Window size for moving average
window_size = 5

# Create a Plotly figure
fig = go.Figure()

# Plotting original data: moving average sentiment score
for company in overview_df['Company Ticker'].unique():
    company_df = overview_df[overview_df['Company Ticker'] == company].copy()
    company_df.sort_values('Date', inplace=True)
    company_df['MA Sentiment Score'] = company_df['Sentiment Score'].rolling(window=window_size, min_periods=1).mean()
    fig.add_trace(go.Scatter(x=company_df['Date'], y=company_df['MA Sentiment Score'], mode='lines+markers', name=f"{company} MA Sentiment"))

# Plotting new data: moving average of sentiment score from financial events
for company in polarity_count['Company Ticker'].unique():
    company_df = polarity_count[polarity_count['Company Ticker'] == company]
    company_df.sort_values('Date', inplace=True)
    company_df['MA Sentiment Score'] = company_df['Sentiment Score'].rolling(window=window_size, min_periods=1).mean()
    fig.add_trace(go.Scatter(x=company_df['Date'], y=company_df['MA Sentiment Score'], mode='lines', name=f"{company} Events MA Sentiment"))

# Update layout
fig.update_layout(title='Comparison of Company Sentiment Scores Over Time (Smoothed)',
                  xaxis_title='Date',
                  yaxis_title='Sentiment Score',
                  template='plotly_dark')

# Show the plot
fig.show()


/var/folders/k7/g3wlfyfj7hx88m5g426q6bx80000gn/T/ipykernel_3698/2053276808.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/k7/g3wlfyfj7hx88m5g426q6bx80000gn/T/ipykernel_3698/2053276808.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/k7/g3wlfyfj7hx88m5g426q6bx80000gn/T/ipykernel_3698/2053276808.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [6]:
import pandas as pd
import plotly.graph_objects as go

# Load your monthly values data
dates = pd.date_range(start="2020-01", periods=52, freq='M')
values = [
    2.5, 2.3, 1.5, 0.3, 0.1, 0.6, 1.0, 1.3, 1.4, 1.2, 1.2, 1.4,  # 2020
    1.4, 1.7, 2.6, 4.2, 5.0, 5.4, 5.4, 5.3, 5.4, 6.2, 6.8, 7.0,  # 2021
    7.5, 7.9, 8.5, 8.3, 8.6, 9.1, 8.5, 8.3, 8.2, 7.7, 7.1, 6.5,  # 2022
    6.4, 6.0, 5.0, 4.9, 4.0, 3.0, 3.2, 3.7, 3.7, 3.2, 3.1, 3.4,  # 2023
    3.1, 3.2, 3.5, None  # 2024
]
monthly_df = pd.DataFrame({'Date': dates, 'Value': values})

# Load and prepare your sentiment data
overview_df = pd.read_csv("../overview_files_archive/overview_inflation.csv")
overview_df['Date'] = pd.to_datetime(overview_df['Date'])

# Window size for moving average
window_size = 8

# Create a Plotly figure
fig = go.Figure()

# Plotting the monthly values as a line plot on the left y-axis
fig.add_trace(go.Scatter(
    x=monthly_df['Date'], y=monthly_df['Value'], 
    mode='lines+markers', name='Monthly Values',
    yaxis='y1'  # Left y-axis
))

# Plotting original data: moving average sentiment score on the right y-axis
for company in overview_df['Company Ticker'].unique():
    company_df = overview_df[overview_df['Company Ticker'] == company].copy()
    company_df.sort_values('Date', inplace=True)
    company_df['MA Sentiment Score'] = company_df['Sentiment Score'].rolling(window=window_size, min_periods=1).mean()
    fig.add_trace(go.Scatter(
        x=company_df['Date'], y=company_df['MA Sentiment Score'], 
        mode='lines+markers', name=f"{company} MA Sentiment",
        yaxis='y2'  # Right y-axis
    ))

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Comparison of Monthly Values and Company Sentiment Scores Over Time',
    xaxis_title='Date',
    yaxis=dict(
        title='Monthly Values',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title='MA Sentiment Score',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right'
    ),
    template='plotly_dark'
)

# Show the plot
fig.show()
